In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("Aula Interativa 2 - ML") \
        .getOrCreate()

spark.version

22/08/16 18:42:11 WARN Utils: Your hostname, pcalais-Inspiron-15-7000-Gaming resolves to a loopback address: 127.0.1.1; using 192.168.0.9 instead (on interface wlp3s0)
22/08/16 18:42:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/16 18:42:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.3.0'

In [3]:
titanic_df = spark.read.csv('/home/pcalais/XPE/cientista-dados/aula2/titanic.csv', header='True', inferSchema='True')

titanic_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Age', 'Pclass', 'Fare'], outputCol='features')


In [10]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(labelCol='Survived', featuresCol='features')

In [11]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[assembler, classifier])

In [12]:
train_data, test_data = titanic_df.randomSplit([0.7, 0.3])

In [13]:
predictSurvivedModel = pipeline.fit(train_data)

22/08/16 18:53:35 ERROR Executor: Exception in task 0.0 in stage 6.0 (TID 5)
org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$2948/0x000000084125f040: (struct<Age:double,Pclass_double_VectorAssembler_3ee51b76ae0a:double,Fare:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:177)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.co

Py4JJavaError: An error occurred while calling o108.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 5) (192.168.0.9 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$2948/0x000000084125f040: (struct<Age:double,Pclass_double_VectorAssembler_3ee51b76ae0a:double,Fare:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:177)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apach

In [15]:
mean_age = titanic_df.agg({'Age': 'mean'}).collect()[0][0]
mean_age

29.69911764705882

In [16]:
titanic_df = titanic_df.fillna(mean_age, subset=['Age'])

In [18]:
train_data, test_data = titanic_df.randomSplit([0.7, 0.3])
predictSurvivedModel = pipeline.fit(train_data)

In [24]:
predictions = predictSurvivedModel.transform(test_data)
predictions.select('passengerId', 'rawPrediction', 'prediction').show(5)



+-----------+-------------+----------+
|passengerId|rawPrediction|prediction|
+-----------+-------------+----------+
|          4|  [52.0,91.0]|       1.0|
|         10|  [52.0,91.0]|       1.0|
|         11|    [0.0,8.0]|       1.0|
|         19|  [55.0,20.0]|       0.0|
|         24|  [52.0,91.0]|       1.0|
+-----------+-------------+----------+
only showing top 5 rows



In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

accuracy



0.6444444444444445